In [2]:
import pandas as pd
import numpy as np



In [3]:
df = pd.read_csv(
    r"C:\Users\Maryam Daniyal\Downloads\logistics\DataCoSupplyChainDataset.csv",
    encoding="latin1"
)

df.head()


,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Delivery Status,Late_delivery_risk,Category Id,Category Name,Customer City,...,Order Zipcode,Product Card Id,Product Category Id,Product Description,Product Image,Product Name,Product Price,Product Status,shipping date (DateOrders),Shipping Mode
0,DEBIT,3,4,91.250000,314.640015,Advance shipping,0,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,2/3/2018 22:56,Standard Class
1,TRANSFER,5,4,-249.089996,311.359985,Late delivery,1,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/18/2018 12:27,Standard Class
2,CASH,4,4,-247.779999,309.720001,Shipping on time,0,73,Sporting Goods,San Jose,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/17/2018 12:06,Standard Class
3,DEBIT,3,4,22.860001,304.809998,Advance shipping,0,73,Sporting Goods,Los Angeles,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/16/2018 11:45,Standard Class
4,PAYMENT,2,4,134.210007,298.250000,Advance shipping,0,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/15/2018 11:24,Standard Class


In [4]:
df.shape
df.columns


Index(['Type', 'Days for shipping (real)', 'Days for shipment (scheduled)',
       'Benefit per order', 'Sales per customer', 'Delivery Status',
       'Late_delivery_risk', 'Category Id', 'Category Name', 'Customer City',
       'Customer Country', 'Customer Email', 'Customer Fname', 'Customer Id',
       'Customer Lname', 'Customer Password', 'Customer Segment',
       'Customer State', 'Customer Street', 'Customer Zipcode',
       'Department Id', 'Department Name', 'Latitude', 'Longitude', 'Market',
       'Order City', 'Order Country', 'Order Customer Id',
       'order date (DateOrders)', 'Order Id', 'Order Item Cardprod Id',
       'Order Item Discount', 'Order Item Discount Rate', 'Order Item Id',
       'Order Item Product Price', 'Order Item Profit Ratio',
       'Order Item Quantity', 'Sales', 'Order Item Total',
       'Order Profit Per Order', 'Order Region', 'Order State', 'Order Status',
       'Order Zipcode', 'Product Card Id', 'Product Category Id',
       'Product De

## 3. Business Context & Problem Definition

Logistics and fleet-based companies operate under tight delivery timelines.
Delays, cancellations, and inefficient shipping directly impact:

- Customer satisfaction
- Operational cost
- Revenue and profitability
- Risk exposure across regions

This project focuses on **delivery performance monitoring and risk detection**
using historical order and shipment data.

The goal is to:
- Identify delivery delays and late shipment risks
- Track logistics performance across regions and shipping modes
- Build automated monitoring logic that can later be operationalized using n8n


## 4. Key Operational KPIs Derived from the Dataset

Based on the available columns, the following KPIs are defined:

### Delivery Performance KPIs
- **On-time delivery rate**
- **Late delivery rate**
- **Average delivery delay (days)**

### Risk Indicators
- **Late delivery risk flag**
- **Delivery cancellations**
- **Negative profit orders**

### Operational Dimensions
- Region
- Market
- Shipping mode
- Product category


In [7]:
# 4.1 Delivery Delay Calculation
# Delivery delay is computed as the difference between: Actual shipping days and Scheduled shipping days
# A positive value indicates a delay.

df['delivery_delay_days'] = (
    df['Days for shipping (real)'] - df['Days for shipment (scheduled)']
)

df[['Days for shipping (real)', 
    'Days for shipment (scheduled)', 
    'delivery_delay_days']].head()


,Days for shipping (real),Days for shipment (scheduled),delivery_delay_days
0,3,4,-1
1,5,4,1
2,4,4,0
3,3,4,-1
4,2,4,-2


**Interpretation: Delivery Delay**

Negative delivery delay values indicate that a large proportion of orders
are shipped **earlier than the scheduled date**.

However, the presence of positive delays confirms operational inconsistency,
where some orders exceed expected delivery timelines.


In [8]:
# 4.2 On-Time vs Late Delivery Rate
# Delivery performance is categorized using the `Delivery Status` field. This allows monitoring of operational efficiency at scale.

delivery_status_counts = (
    df['Delivery Status']
    .value_counts(normalize=True)
    .reset_index()
)

delivery_status_counts.columns = ['Delivery Status', 'Proportion']
delivery_status_counts



,Delivery Status,Proportion
0,Late delivery,0.548291
1,Advance shipping,0.230402
2,Shipping on time,0.178352
3,Shipping canceled,0.042954


**Interpretation: Delivery Status Distribution**

More than half of all orders are categorized as **late deliveries**,
while fewer than 20% are delivered strictly on time.

This indicates systemic delivery performance issues rather than isolated delays.


In [9]:
# 4.3 Late Delivery Risk Validation
# The dataset includes a precomputed binary indicator `Late_delivery_risk`. We validate whether this flag aligns with actual delivery delays.
 

df.groupby('Late_delivery_risk')['delivery_delay_days'].mean()



Late_delivery_risk
0   -0.711584
1    1.618184
Name: delivery_delay_days, dtype: float64

**Interpretation: Delivery Status Distribution**

More than half of all orders are categorized as **late deliveries**,
while fewer than 20% are delivered strictly on time. This indicates systemic delivery performance issues rather than isolated delays.


In [10]:
# 4.4 Regional Delivery Risk Analysis
# To identify high-risk operational areas, late delivery risk is analyzed by region.

region_risk = (
    df.groupby('Order Region')['Late_delivery_risk']
    .mean()
    .sort_values(ascending=False)
)

region_risk.head(10)



Order Region
Central Africa    0.579606
South Asia        0.562670
East Africa       0.559395
Western Europe    0.558486
South of  USA     0.557726
Eastern Europe    0.556633
East of USA       0.556616
Southeast Asia    0.555299
Central Asia      0.553345
West Asia         0.552837
Name: Late_delivery_risk, dtype: float64

**Interpretation: Regional Delivery Risk**

Late delivery risk is consistently high across multiple regions,
with Central Africa and South Asia showing the highest exposure.

This suggests that delivery delays are geographically systemic
and likely driven by infrastructure, logistics complexity, or routing issues.


## 5. Automation Objective

Based on the analysis, this project implements a **weekly automated logistics
performance and risk monitoring system**.

The system is designed to:
- Track delivery delays and late shipment risk
- Identify high-risk regions
- Generate a concise weekly operational summary
- Support decision-making for logistics and fleet operations teams


## 6. Weekly KPI Preparation for Automation

To enable automated monitoring, KPIs are aggregated at a **weekly level**.
Date fields are first parsed and standardized.


In [11]:
# Parse order date
df['order_date'] = pd.to_datetime(df['order date (DateOrders)'])

# Create year-week identifier
df['year_week'] = df['order_date'].dt.strftime('%Y-%U')

df[['order_date', 'year_week']].head()


,order_date,year_week
0,2018-01-31 22:56:00,2018-04
1,2018-01-13 12:27:00,2018-01
2,2018-01-13 12:06:00,2018-01
3,2018-01-13 11:45:00,2018-01
4,2018-01-13 11:24:00,2018-01


### Weekly KPI Definitions

The following KPIs are calculated weekly:
- Late delivery rate
- Average delivery delay (days)
- Cancellation rate
- Regional risk exposure

In [12]:
weekly_kpis = (
    df.groupby('year_week')
    .agg(
        late_delivery_rate=('Late_delivery_risk', 'mean'),
        avg_delivery_delay=('delivery_delay_days', 'mean'),
        cancellation_rate=('Order Status', lambda x: (x == 'CANCELED').mean()),
        total_orders=('Order Id', 'count')
    )
    .reset_index()
)

weekly_kpis.head()


,year_week,late_delivery_rate,avg_delivery_delay,cancellation_rate,total_orders
0,2015-00,0.518962,0.467066,0.013972,501
1,2015-01,0.536709,0.487764,0.010970,1185
2,2015-02,0.542668,0.633803,0.019056,1207
3,2015-03,0.529752,0.531405,0.019008,1210
4,2015-04,0.564397,0.537326,0.014766,1219


### Weekly High-Risk Region Identification

To support targeted intervention, the region with the highest
late delivery risk is identified each week.


In [13]:
weekly_region_risk = (
    df.groupby(['year_week', 'Order Region'])['Late_delivery_risk']
    .mean()
    .reset_index()
)

top_region_per_week = (
    weekly_region_risk
    .sort_values(['year_week', 'Late_delivery_risk'], ascending=[True, False])
    .groupby('year_week')
    .first()
    .reset_index()
)

top_region_per_week.head()


,year_week,Order Region,Late_delivery_risk
0,2015-00,Central America,0.528226
1,2015-01,Central America,0.583851
2,2015-02,South America,0.574176
3,2015-03,South America,0.544379
4,2015-04,Caribbean,0.601266


### Final Weekly KPI Table for Automation


In [14]:
final_weekly_kpis = pd.merge(
    weekly_kpis,
    top_region_per_week,
    on='year_week',
    how='left'
)

final_weekly_kpis.rename(
    columns={
        'Order Region': 'top_risk_region',
        'Late_delivery_risk': 'top_region_risk_rate'
    },
    inplace=True
)

final_weekly_kpis.head()


,year_week,late_delivery_rate,avg_delivery_delay,cancellation_rate,total_orders,top_risk_region,top_region_risk_rate
0,2015-00,0.518962,0.467066,0.013972,501,Central America,0.528226
1,2015-01,0.536709,0.487764,0.010970,1185,Central America,0.583851
2,2015-02,0.542668,0.633803,0.019056,1207,South America,0.574176
3,2015-03,0.529752,0.531405,0.019008,1210,South America,0.544379
4,2015-04,0.564397,0.537326,0.014766,1219,Caribbean,0.601266


### Export Weekly KPIs for n8n Automation

The final KPI table is exported as a CSV file for downstream automation.


In [16]:
import os

# Create outputs directory if it does not exist
os.makedirs('outputs', exist_ok=True)


In [ ]:
    final_weekly_kpis.to_csv(
        'outputs/weekly_kpis.csv',
        index=False
    )


In [18]:
import os

os.listdir('outputs')


['weekly_kpis.csv']